In [ ]:
# 

In [ ]:

from time import time
import pandas as pd
import numpy as np
import re
import sys
import csv
csv.field_size_limit(sys.maxsize)

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import itertools
import datetime

import pprint
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#being able to read csv stored in google drive 
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Reading the dataset with no columns titles and with latin encoding 
df_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweets.csv', delimiter=';', skiprows=0, lineterminator='\n' )
df_raw.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [ ]:
 # As the data has no column titles, we will add our own
df_raw.columns = ["id", "user", "fullname", "url", "timestamp", "replies","likes","retweets","text"]

# Show the first 5 rows of the dataframe.
# You can specify the number of rows to be shown as follows: df_raw.head(10)
df_raw.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text
0,1.132977e+18,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [ ]:

# Ommiting every column except for the text and the date, as we won't need any of the other information
df = df_raw[['timestamp','text']]
df.sample(5)

,timestamp,text
13899466,2019-10-04 17:42:23+00,$ZRX. Push! Push! Push! Its Rising! $BTC marke...
12704221,2019-09-16 04:55:05+00,🔵 El bitcoin traerá consecuencias devastadoras...
4843375,2019-05-29 14:57:57+00,Dm for safe secure and guarantee investment @B...
4097033,2018-04-07 19:30:13+00,1 #bitcoin está custando R$23990.98 na FoxBit....
9186367,2019-07-26 16:15:28+00,$xojenn2 @Kiddanny8\r


In [ ]:
#Create split by dates 
df.timestamp.dtypes

dtype('O')

In [ ]:
df['date'] = pd.to_datetime(df['timestamp'],format= '%Y-%m-%d').dt.date
df['date']

0           2019-05-27
1           2019-05-27
2           2019-05-27
3           2019-05-27
4           2019-05-27
               ...    
16889760    2019-11-23
16889761    2019-11-23
16889762    2019-11-23
16889763    2019-11-23
16889764    2019-11-23
Name: date, Length: 16889765, dtype: object

In [ ]:
#new df is now SORTED by DATE
df = df.sort_values(by='date')
df.head()

,timestamp,text,date
11291156,2007-04-19 07:14:38+00,is happily mugging at BTC where she will hook ...,2007-04-19
21,2009-01-11 03:33:52+00,Running bitcoin\r,2009-01-11
5164467,2009-01-21 17:29:40+00,Looking at ways to add more anonymity to bitco...,2009-01-21
5210398,2009-01-27 20:14:10+00,Thinking about how to reduce CO2 emissions fro...,2009-01-27
5269667,2009-01-29 13:37:53+00,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...,2009-01-29


In [ ]:
df = df.set_index('date')
df.head()

,timestamp,text
date,,
2007-04-19,2007-04-19 07:14:38+00,is happily mugging at BTC where she will hook ...
2009-01-11,2009-01-11 03:33:52+00,Running bitcoin\r
2009-01-21,2009-01-21 17:29:40+00,Looking at ways to add more anonymity to bitco...
2009-01-27,2009-01-27 20:14:10+00,Thinking about how to reduce CO2 emissions fro...
2009-01-29,2009-01-29 13:37:53+00,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...


In [ ]:
# df.date.year > '2017'
startdate = pd.to_datetime("2017-01-01").date()
df = df.loc[startdate:]
df.tail()

,timestamp,text
date,,
2019-11-23,2019-11-23 05:47:01+00,Bitcoin:$7231.27732436\nEthereum:$150.48909214...
2019-11-23,2019-11-23 05:40:05+00,BTC\n\nfinexの乖離がまだ異常水域です。\n\nペナントの収束タイミングに合わせた...
2019-11-23,2019-11-23 05:40:05+00,"#BTC New Hub, we have relaunch our #NEWYORKCOI..."
2019-11-23,2019-11-23 05:39:55+00,トランザクション収入がまともにないBitcoinさんのマイナーは単価に依存するからこうなるん...
2019-11-23,2019-11-23 15:45:06+00,$BTC - an update on the longer term view for B...


In [ ]:
!pip install whatthelang

In [ ]:
from whatthelang import WhatTheLang
wtl = WhatTheLang()
result = [wtl.predict_lang(row) for row in df['text']]
df['lang'] = result
df.head()

,timestamp,text,lang
date,,,
2017-01-01,2017-01-01 10:30:10+00,$975.00 at 11:30 UTC [24h Range: $950.81 - $97...,en
2017-01-01,2017-01-01 10:30:06+00,#Bitcoin\nUltima: R$ 3440.00 Alta: R$ 3440.00 ...,en
2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00@bitstamp. High $...,en
2017-01-01,2017-01-01 10:30:06+00,#8BitCoin #8BIT $0.011464 (0.86%) 0.00001195 B...,en
2017-01-01,2017-01-01 10:23:44+00,#Bitcoin last trade\n\n@bitfinex $976.81\n@btc...,en


In [ ]:
#ATTENTION, we will take only text, date and lang column FOR THE texts in ENglish and below DATES starting 2017 jan
en_df = df[df["lang"] == 'en']
en_df.sample(10, random_state = 5)

,timestamp,text,lang
date,,,
2019-10-21,2019-10-21 08:31:09+00,Chainlink is gaining fast.\n\nhttps://t.co/LQS...,en
2019-06-21,2019-06-21 21:02:12+00,Bitcoin turned 10 in January. A decade after S...,en
2019-06-07,2019-06-07 09:15:22+00,RT wiz: I can see a future where everyone has ...,en
2019-09-19,2019-09-19 14:28:59+00,If Bitcoin ETF's are being rejected.. what are...,en
2019-09-15,2019-09-15 20:52:33+00,@NYC | Immediate Hire - Customer Service Oppor...,en
2018-03-26,2018-03-26 20:07:47+00,#BTC Average: 7981.22$\n\n#Bitfinex - 7910.00$...,en
2019-09-20,2019-09-20 18:01:18+00,₿ Actual #Bitcoin #BTC price is \n1 BTC = 9.20...,en
2019-11-06,2019-11-06 17:06:30+00,Explainer: Central Bank Digital Currencies: Ed...,en
2019-05-16,2019-05-16 03:30:39+00,https://t.co/6Szi9FmjcK\n\nBTC/USD - HEDEF BÖL...,en


In [ ]:
# checking for null values, if any
en_df.isnull().sum()

timestamp    0
text         0
lang         0
dtype: int64

In [ ]:
#ditching all row when text is null, as need text for analysis
en_df.dropna(how='any', inplace=True)
en_df.sample(3)

,timestamp,text,lang
date,,,
2019-07-09,2019-07-09 19:12:17+00,The only chart I care about. RIP bcash via /r/...,en
2019-08-22,2019-08-22 15:55:06+00,Yeah don't listen to this clown.\r,en
2019-09-05,2019-09-05 01:30:20+00,$Bitcoin https://t.co/BYWuBBJUEp https://t.co/...,en


In [ ]:
en_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12343033 entries, 2017-01-01 to 2019-11-23
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  object
 1   text       object
 2   lang       object
dtypes: object(3)
memory usage: 376.7+ MB


In [ ]:
# import pickle
# en_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsENdates.pkl')
#  to load your pickle file to a new notebook and use the dataframe:
# with open("/content/drive/MyDrive/Colab Notebooks/NLP/tweetsClean.pkl", "rb") as file:
#         df = pickle.load(file)

In [ ]:
#export to new CSV as it crashes otherwise
en_df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsENdates.csv')